In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [2]:
df = pd.read_csv('/content/diabetes.csv')

In [4]:
df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [5]:
df.corr()['Outcome']

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [6]:
X = df.iloc[:,:-1].values
y = df.iloc[:,-1].values

In [7]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X = scaler.fit_transform(X)

In [8]:
X.shape

(768, 8)

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state=1)

In [69]:
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Dropout

In [13]:
from keras.src.engine.training import optimizer
model = Sequential()
model.add(Dense(32, activation='relu', input_dim = 8))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='Adam',loss='binary_crossentropy', metrics=['accuracy'])

In [16]:
model.fit(X_train,y_train, batch_size=32, epochs=100, validation_data=(X_test,y_test))

Epoch 1/100
20/20 [==============================] - 0s 8ms/step - loss: 0.4561 - accuracy: 0.7704 - val_loss: 0.4642 - val_accuracy: 0.8052
Epoch 2/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4548 - accuracy: 0.7671 - val_loss: 0.4636 - val_accuracy: 0.8052
Epoch 3/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4537 - accuracy: 0.7736 - val_loss: 0.4635 - val_accuracy: 0.8052
Epoch 4/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4526 - accuracy: 0.7736 - val_loss: 0.4633 - val_accuracy: 0.8052
Epoch 5/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4513 - accuracy: 0.7752 - val_loss: 0.4621 - val_accuracy: 0.8117
Epoch 6/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4510 - accuracy: 0.7704 - val_loss: 0.4617 - val_accuracy: 0.8052
Epoch 7/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4497 - accuracy: 0.7704 - val_loss: 0.4616 - val_accuracy: 0.8052
Epoch 8/100
2

In [18]:
!pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.5/129.5 kB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 14.7 MB/s eta 0:00:00


In [20]:
# How to select appropriate optimizer
import kerastuner as kt

Using TensorFlow backend


<ipython-input-20-eaada7debf89>:2: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


In [26]:
def build_model(hp):
  model = Sequential()
  model.add(Dense(32, activation='relu', input_dim=8))
  model.add(Dense(1, activation='sigmoid'))
  optimizer = hp.Choice('optimizer',['adam','sgd','rmsprop','adadelta'])
  model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
  return model

In [27]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=5)

Reloading Tuner from ./untitled_project/tuner0.json


In [28]:
tuner.search(X_train,y_train, epochs=5, validation_data=(X_test,y_test))

Trial 4 Complete [00h 00m 02s]
val_accuracy: 0.8051947951316833

Best val_accuracy So Far: 0.8051947951316833
Total elapsed time: 00h 00m 29s


In [29]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'adam'}

In [30]:
model = tuner.get_best_models(num_models=1)[0]
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                288       
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 321 (1.25 KB)
Trainable params: 321 (1.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [34]:
model.fit(X_train,y_train, batch_size=32, epochs=100,validation_data=(X_test,y_test))

Epoch 1/100
20/20 [==============================] - 2s 11ms/step - loss: 0.5792 - accuracy: 0.7394 - val_loss: 0.5450 - val_accuracy: 0.7857
Epoch 2/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5537 - accuracy: 0.7541 - val_loss: 0.5264 - val_accuracy: 0.7857
Epoch 3/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5331 - accuracy: 0.7671 - val_loss: 0.5113 - val_accuracy: 0.7857
Epoch 4/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5175 - accuracy: 0.7655 - val_loss: 0.5003 - val_accuracy: 0.7922
Epoch 5/100
20/20 [==============================] - 0s 3ms/step - loss: 0.5055 - accuracy: 0.7720 - val_loss: 0.4929 - val_accuracy: 0.7922
Epoch 6/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4970 - accuracy: 0.7785 - val_loss: 0.4869 - val_accuracy: 0.7922
Epoch 7/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4891 - accuracy: 0.7801 - val_loss: 0.4850 - val_accuracy: 0.7857
Epoch 8/100


In [35]:
# How to select appropriate optimizer

In [50]:
def build_model(hp):
  model = Sequential()
  units = hp.Int('units',min_value=8,max_value=128,step=8)
  model.add(Dense(units=units, activation='relu', input_dim=8))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])
  return model

In [51]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=5,
                        directory='unit_dir',
                        project_name='keras_units')

In [52]:
tuner.search(X_train,y_train, epochs=5, validation_data=(X_test,y_test))

Trial 5 Complete [00h 00m 02s]
val_accuracy: 0.8116883039474487

Best val_accuracy So Far: 0.8116883039474487
Total elapsed time: 00h 00m 09s


In [54]:
tuner.get_best_hyperparameters()[0].values

{'units': 128}

In [70]:
# layers, units, optmizer
def build_model(hp):
  model = Sequential()
  counter = 0
  for i in range(hp.Int('num_layers',min_value=1,max_value=10)):
    if counter == 0:
      model.add(Dense(hp.Int('units'+str(i), min_value=8, max_value=128,step=8),
                      input_dim=8,
                      activation=hp.Choice('activation'+str(i),values=['relu','tanh','sigmoid'])))
      model.add(Dropout(hp.Choice('dropout'+str(i),values=[0.1,0.2,0.3,0.4,0.5,0.6,0.8,0.9])))
    else:
      model.add(Dense(hp.Int('units'+str(i), min_value=8, max_value=128,step=8),
                      activation=hp.Choice('activation'+str(i),values=['relu','tanh','sigmoid'])))

    counter += 1

    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer=hp.Choice('optimizer',values=['rmsprop','adam','nadam']),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

In [71]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=5,
                        directory='newdir1',
                        project_name='final')

In [72]:
tuner.search(X_train,y_train, epochs=5, validation_data=(X_test,y_test))

Trial 5 Complete [00h 00m 02s]
val_accuracy: 0.798701286315918

Best val_accuracy So Far: 0.798701286315918
Total elapsed time: 00h 00m 10s


In [73]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 1,
 'units0': 72,
 'activation0': 'relu',
 'dropout0': 0.1,
 'optimizer': 'adam'}

In [74]:
model = tuner.get_best_models(num_models=1)[0]

In [75]:
model.fit(X_train,y_train, epochs=100, validation_data=(X_test,y_test))

Epoch 1/100
20/20 [==============================] - 1s 12ms/step - loss: 0.5098 - accuracy: 0.7622 - val_loss: 0.4965 - val_accuracy: 0.7987
Epoch 2/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4940 - accuracy: 0.7671 - val_loss: 0.4849 - val_accuracy: 0.7922
Epoch 3/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4839 - accuracy: 0.7687 - val_loss: 0.4763 - val_accuracy: 0.7857
Epoch 4/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4788 - accuracy: 0.7671 - val_loss: 0.4711 - val_accuracy: 0.7922
Epoch 5/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4689 - accuracy: 0.7752 - val_loss: 0.4679 - val_accuracy: 0.7987
Epoch 6/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4689 - accuracy: 0.7655 - val_loss: 0.4659 - val_accuracy: 0.7922
Epoch 7/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4618 - accuracy: 0.7769 - val_loss: 0.4630 - val_accuracy: 0.7922
Epoch 8/100
